In [1]:
import os
import json
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI

from website import Website

/Users/priyaraja/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
link_system_prompt = "You are provided with a list of links found on a Italian restaurant webpage. \
You are able to decide which of the links would be most relevant to include in the restaurant menu, \
such as links to an menu pdf file, Menù page, Piatti, or Bevande.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "menu pdf", "url": "https://www.ristoranteapprodo.com/Documenti/MenuEstivo2024.pdf"},
        {"type": "menu page", "url": "https://www.giocapizza.com/men%C3%B9"}
    ]
}
"""

In [4]:
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")

MODEL = 'gpt-4o-mini'
openai = OpenAI()

ed = Website("https://www.giocapizza.com/")
print(ed.links)

API key looks good so far
['https://www.giocapizza.com', 'tel:349-6705657', 'https://www.instagram.com/giocapizza/', 'https://www.facebook.com/giocapizza/', 'https://www.tripadvisor.it/Restaurant_Review-g2337656-d17784755-Reviews-Gioca_Pizza-Adrara_San_Martino_Province_of_Bergamo_Lombardy.html', 'https://www.youtube.com/@GiocaPizza', 'https://www.pinterest.jp/giocapizza/', 'https://www.giocapizza.com', 'https://www.giocapizza.com/incorniciate', 'https://www.giocapizza.com/menù', 'https://www.giocapizza.com/servizi', 'https://www.giocapizza.com/menù', 'https://www.giocapizza.com/incorniciate', 'https://www.giocapizza.com/incorniciate', 'https://www.giocapizza.com/incorniciate', 'mailto:giocapizza@gmail.com', 'http://www.sinapsisnc.com']


In [5]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the italian restaurant website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for the restaurant menu, respond with the full https URL in JSON format."
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt


In [6]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)


In [7]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result


In [8]:
system_prompt = "You are an assistant that analyzes the contents of several menu pages from an italian restaurant website \
and creates restaurant menu with dishes and prices in Euro. Respond in markdown."

def get_restaurant_menu_user_prompt(company_name, url):
    user_prompt = f"You are looking at a restaurant called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a restaurant menu in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt


In [9]:
def create_restaurant_menu(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_restaurant_menu_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [10]:
create_restaurant_menu("La Cascina", "https://www.lacascinacredaro.it/")

Found links: {'links': [{'type': 'piatti', 'url': 'http://www.byserviziinternet.com/cascina/#piatti'}]}


# La Cascina Menu

## Antipasti
- **Bruschetta Classica** - €6.00  
  Toasted bread with fresh tomatoes, basil, and olive oil.
  
- **Antipasto Misto** - €10.00  
  A selection of cured meats and cheeses.

- **Carciofi alla Giudia** - €8.00  
  Deep-fried artichokes served with lemon.

## Primi Piatti
- **Gnocchetti con Erbette** - €12.00  
  Homemade gnocchi with garden greens.

- **Paccheri con Polipetti** - €14.00  
  Wide pasta with baby octopus in tomato sauce.

- **Risotto ai Frutti di Mare** - €16.00  
  Creamy risotto with assorted seafood.

- **Tagliolini al Tartufo** - €15.00  
  Homemade tagliolini pasta with truffle sauce.

## Secondi Piatti
- **Bistecca alla Fiorentina** - €25.00  
  Grilled T-bone steak from Tuscany.

- **Cinghiale in Umido** - €18.00  
  Stewed wild boar with red wine and herbs.

- **Branzino al forno** - €20.00  
  Oven-roasted sea bass with herbs.

## Pizze
- **Margherita** - €8.00  
  Classic pizza with tomato, mozzarella, and basil.

- **Diavola** - €10.00  
  Spicy salami and mozzarella.

- **Quattro Formaggi** - €12.00  
  Four cheese pizza: mozzarella, gorgonzola, parmesan, and fontina.

## Contorni
- **Patate al forno** - €5.00  
  Oven-baked potatoes with rosemary.

- **Insalata Verde** - €4.00  
  Mixed green salad with vinaigrette.

## Dessert
- **Tiramisù** - €6.00  
  Classic Italian coffee-flavored dessert.

- **Panna Cotta** - €5.00  
  Creamy dessert topped with berry sauce.

## Bevande
- **Vino della casa** - €10.00 (bottle)  
  House wine, red or white.

- **Acqua minerale** - €2.00  
  Sparkling or still mineral water.

- **Caffè** - €1.50  
  Italian espresso.

## Note
- *Chiuso il mercoledì* (Closed on Wednesdays).
- For reservations, please call +39 035 936383 or visit us at Via L. Cadorna, 9, 24060 Credaro (BG). 

*Enjoy your meal at La Cascina!*